## Analyze whether SNWD varies more from year to year or from place to place.

In [11]:
import pandas as pd
import numpy as np
import urllib
import math

In [13]:
import findspark
findspark.init()

from pyspark import SparkContext
sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [14]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [15]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSSBBSS'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
393
+------------------+------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+------------------+-----------+-----------+------+--------------------+------+
|           coeff_1|           coeff_2|           coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|              res_3|          res_mean|    station|  total_var|undefs|              vector|  year|
+------------------+------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+------------------+-----------+-----------+------+--------------------+------+
|11183.467759022718|2476.4223902765893| 385.1362997238969|   2493.3|SSSSBBSS| 33.9833|  -109.75|       SNWD|0.09903383540055485|0.05733362444014835|0.05647398712192465|0.969514518488140

In [16]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [44]:
#extract longitude and latitude for each station
feature='coeff_3'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_3'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_3 FROM weather


,station,year,coeff_3
0,USC00023926,1979.0,385.136300
1,USC00023926,1982.0,-689.642002
2,USC00023926,1983.0,-1692.249084
3,USC00023926,1973.0,-378.367818
4,USC00028326,1973.0,1898.158307


In [45]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_3')
year_station_table.head(10)

station,USC00020159,USC00020170,USC00020174,USC00020855,USC00021749,USC00022779,USC00023082,USC00023448,USC00023505,USC00023683,...,USC00028162,USC00028326,USC00029271,USC00029367,USC00029622,USC00294375,USC00295273,USC00297191,USC00297386,USW00093139
year,,,,,,,,,,,,,,,,,,,,,
1905.0,NaN,NaN,NaN,-122.892273,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-328.131300,NaN,NaN,NaN,NaN,NaN
1909.0,NaN,NaN,NaN,-154.523119,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-72.744722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1915.0,102.714449,NaN,NaN,-55.939499,NaN,NaN,NaN,NaN,NaN,-401.793166,...,106.325439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1916.0,NaN,NaN,NaN,42.990105,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1917.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1930.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,88.411975,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1932.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-61.029943,NaN,NaN,NaN,NaN,NaN
1934.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [42]:
def RMS(Mat):
    return (np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total MS                   =',RMS(year_station_table)
print 'MS removing mean-by-station= {}, fraction explained= {}'.format(RMS(tbl_minus_station),round(100*(1-RMS(tbl_minus_station)/RMS(year_station_table)),1))
print 'MS removing mean-by-year   = {}, fraction explained= {}'.format(RMS(tbl_minus_year),round(100*(1-RMS(tbl_minus_year)/RMS(year_station_table)),1))

total MS                   = 497559.927705
MS removing mean-by-station= 414501.987562, fraction explained= 16.7
MS removing mean-by-year   = 251941.741568, fraction explained= 49.4


In [26]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 497559.927705
0 after removing mean by year    = 251941.741568
0 after removing mean by stations= 214953.136903
1 after removing mean by year    = 208174.883863
1 after removing mean by stations= 206179.54399
2 after removing mean by year    = 205430.1809
2 after removing mean by stations= 205105.430407
3 after removing mean by year    = 204950.0629
3 after removing mean by stations= 204870.413163
4 after removing mean by year    = 204827.661179
4 after removing mean by stations= 204804.036392
